In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


file_path = r"C:\Users\chris\Corporate_Environmental_Impact\data\raw\Scope-3-Environmental-Impact-Data-2022.xlsx"

df = pd.read_excel(file_path, sheet_name="0%")

# Prüfen ob es fehlende Werte gibt
print(df.isnull().sum())

# Zeilen mit fehlenden Werten entfernen
df=df.dropna()

# Nicht relevante Spalten entfernen (Angaben ob Wert geschätzt oder für ML verwendet wurde)
df = df.loc[:, ~df.columns.str.contains('Imputed|Set', case=False)]
print(df.columns)

# Nach Auffälligkeiten in den numerischen Wertenschauen
df.describe()


In [ ]:
min_emission=df['Total GHG Environmental Impact (Scope 1, 2, 3)'].min()
print(f"{min_emission:,.2f}")

max_emission=df['Total GHG Environmental Impact (Scope 1, 2, 3)'].max()
print(f"{max_emission:,.2f}")

# Zeile mit größtem GHG Impact
min_row = df[df['Total GHG Environmental Impact (Scope 1, 2, 3)'] == min_emission]

# Zeile mit niedrigstem GHG Impact
max_row = df[df['Total GHG Environmental Impact (Scope 1, 2, 3)'] == max_emission]

min_row
max_row


In [ ]:
# Umwandlung in positive GHG-Werte, Gruppiert nach Industry/Branche
top_10_industries = df.groupby('Industry (Exiobase)')['Total GHG Environmental Impact (Scope 1, 2, 3)'].mean().abs().sort_values(ascending=False).head(10)

# Balkendiagramm erstellen
plt.figure(figsize=(13, 8)) 
plt.style.use('seaborn-v0_8')

bars = plt.barh(
    top_10_industries.index, 
    top_10_industries, 
)
# Werte im Balken anzeigen
for bar in bars:
    width = bar.get_width()
    formatted_value = f'{width:,.0f}'
    plt.text(width / 2, bar.get_y() + bar.get_height()/2, formatted_value, va='center', ha='center', fontsize=10, color='black')

plt.xlabel('Durchschnittlicher GHG Impact (Scope 1, 2, 3) in USD')
plt.title('Top 10 Branchen mit dem höchsten durchschnittlichen GHG Impact 2012-2020', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('top_10_indrusties_emissions.png')
plt.show()



In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Spaltennamen bereinigen
df.columns = df.columns.str.strip()

# relevante Spalten kopieren
df_scope = df[['Industry (Exiobase)', 'Scope 1', 'Scope 2', 'Scope 3']].copy()

# In positive Werte umwandeln
df_scope[['Scope 1', 'Scope 2', 'Scope 3']] = df_scope[['Scope 1', 'Scope 2', 'Scope 3']].abs()

# Gruppieren und Durchschnitt berechnen
mean_by_industry = df_scope.groupby('Industry (Exiobase)').mean()

# Gesamt-Impact berechnen
mean_by_industry['Total'] = mean_by_industry[['Scope 1', 'Scope 2', 'Scope 3']].sum(axis=1)

# Top 10 Industrien mit höchstem durchschnittlichem GHG Impact
top_10 = mean_by_industry.sort_values(by='Total', ascending=False).head(10)

# Fürs gestapelte Balkendiagramm vorbereiten
industries = top_10.index
scope1 = top_10['Scope 1']
scope2 = top_10['Scope 2']
scope3 = top_10['Scope 3']

# Abbildung erstellen
plt.figure(figsize=(13, 8))
plt.style.use('seaborn-v0_8')

bar1 = plt.barh(industries, scope1, label='Scope 1')
bar2 = plt.barh(industries, scope2, left=scope1, label='Scope 2')
bar3 = plt.barh(industries, scope3, left=scope1 + scope2, label='Scope 3')

# Werte im Balken anzeigen (optional)
for i in range(len(industries)):
    total = scope1[i] + scope2[i] + scope3[i]
    plt.text(total / 2, i, f'{int(total):,}', va='center', ha='center', fontsize=10, color='black')

# Achsentitel und Legende
plt.xlabel('Durchschnittlicher GHG Impact in USD')
plt.title('Top 10 Branchen nach GHG Impact (aufgeteilt nach Scope 1, 2, 3)', fontsize=12)
plt.legend()
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('top_10_branches_scopes_stacked.png')
plt.show()


In [ ]:
# print(top_10_industries)

In [ ]:
# Umwandlung in positive GHG-Werte, Gruppiert nach Industry/Branche
top_10_low_emissions = df.groupby('Industry (Exiobase)')['Total GHG Environmental Impact (Scope 1, 2, 3)'].mean().abs().sort_values(ascending=True).head(10)

# Balkendiagramm erstellen
plt.figure(figsize=(15, 8)) 
plt.style.use('seaborn-v0_8')

bars = plt.barh(
    top_10_low_emissions.index, 
    top_10_low_emissions, 
)
# Werte im Balken anzeigen
for bar in bars:
    width = bar.get_width()
    formatted_value = f'{width:,.0f}'
    plt.text(width / 2, bar.get_y() + bar.get_height()/2, formatted_value, va='center', ha='center', fontsize=10, color='black')

plt.xlabel('Durchschnittlicher GHG Impact (Scope 1, 2, 3) in USD')
plt.title('Top 10 Branchen mit dem niedrigsten durchschnittlichen GHG Impact 2012-2020', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('top_10_low_emission_industries.png')
plt.show()


In [ ]:
print(top_10_low_emissions)

In [ ]:
import plotly.graph_objects as go

# Series in Liste umwandeln
top_10_low_emission_industries = top_10_low_emissions.index.tolist()

# Top 10 Branchen mit niedrigem Impact auswählen
df_filtered = df[df['Industry (Exiobase)'].isin(top_10_low_emission_industries)]

# Gruppieren nach Jahr und Branche, Durchschnitt berechnen
ghg_by_year = df_filtered.groupby(['Year', 'Industry (Exiobase)'])['Total GHG Environmental Impact (Scope 1, 2, 3)'].mean().abs().reset_index()

# Leere Figur für Plotly
fig = go.Figure()

# Für jede Branche eine Linie hinzufügen
for industry in top_10_low_emission_industries:
    data = ghg_by_year[ghg_by_year['Industry (Exiobase)'] == industry]
    fig.add_trace(go.Scatter(
        x=data['Year'], 
        y=data['Total GHG Environmental Impact (Scope 1, 2, 3)'],
        mode='lines',
        name=industry, 
        line=dict(width=2), 
    ))

# Layout
fig.update_layout(
    title='Zeitlicher Verlauf des durchschnittlichen GHG Impacts (2012–2020) – Top 10 Branchen mit niedrigstem Impact',
    xaxis_title='Jahr',
    yaxis_title='Durchschnittlicher GHG Impact in Milliarden (B)',
    template='seaborn', 
    legend_title='Branche',
    legend=dict(
        x=1.05, 
        y=1,
        traceorder='normal',
        bgcolor='rgba(255, 255, 255, 0)',  
        borderwidth=0,
    ),
    showlegend=True 
)
fig.write_html("../ghg_trend_top_10_low_impact.html")
fig.show()


In [ ]:
# photovoltaic production untersuchen 
# Filter erstellen
maske = (df['Industry (Exiobase)'] == 'Production of electricity by solar photovoltaic')

# Gefilterte Daten anzeigen
df_photovoltaic = df[maske]
print(df_photovoltaic)

# 15 Unternehmen insg. -> Hinzukommen neuer Unternehmen sorgt für starken Anstieg

In [ ]:
# Manufacture of textiles (17) untersuchen
# Filter erstellen
maske = (df['Industry (Exiobase)'] == 'Manufacture of textiles (17)')

# Gefilterte Daten anzeigen
df_textiles = df[maske]
print(df_textiles)

# Schwankungen entstehen durch fehlende Kontinuierlichkeit der berichtenden Unternehmen, sehr geringe Anzahl an Unternehmen

In [ ]:
# Untersuche Manufacture of wood and of products of wood and cork, except furniture; manufacture of articles of straw and plaiting materials (20)
maske = (df['Industry (Exiobase)'] == 'Manufacture of wood and of products of wood and cork, except furniture; manufacture of articles of straw and plaiting materials (20)')

# Gefilterte Daten anzeigen
df_wood = df[maske]
print(df_wood)

# Zwei Unternehmen insg. -> sinkende Kosten bei diesen Unternehmen


In [ ]:
# Untersuche Publishing, printing and reproduction of recorded media (22)

# Filter erstellen
maske = (df['Industry (Exiobase)'] == 'Publishing, printing and reproduction of recorded media (22)')

# Gefilterte Daten anzeigen
df_print = df[maske]
print(df_print)

# 152 Unternehmen, kontinuierlicher Abstieg

In [ ]:
# Zeitlicher Verlauf der Emissionen der TOP 10 Branchen mit höchstem Impact

# Series in Liste umwandeln
top_10_high_emission_industries = top_10_industries.index.tolist()

# Top 10 Branchen mit niedrigem Impact auswählen
df_filtered = df[df['Industry (Exiobase)'].isin(top_10_high_emission_industries)]

# Gruppieren nach Jahr und Branche, Durchschnitt berechnen
ghg_by_year = df_filtered.groupby(['Year', 'Industry (Exiobase)'])['Total GHG Environmental Impact (Scope 1, 2, 3)'].mean().abs().reset_index()

# leere Figure
fig = go.Figure()

# Für jede Branche eine Linie hinzufügen
for industry in top_10_high_emission_industries:
    data = ghg_by_year[ghg_by_year['Industry (Exiobase)'] == industry]
    fig.add_trace(go.Scatter(
        x=data['Year'], 
        y=data['Total GHG Environmental Impact (Scope 1, 2, 3)'],
        mode='lines',
        name=industry, 
        line=dict(width=2), 
    ))

# Layout anpassen
fig.update_layout(
    title='Zeitlicher Verlauf des durchschnittlichen GHG Impacts (2012–2020) – Top 10 Branchen mit höchstem Impact',
    xaxis_title='Jahr',
    yaxis_title='Durchschnittlicher GHG Impact in Milliarden (B)',
    template='seaborn', 
    legend_title='Branche',
    legend=dict(
        x=1.05, 
        y=1,
        traceorder='normal',
        bgcolor='rgba(255, 255, 255, 0)',
        borderwidth=0,
    ),
    showlegend=True 
)

fig.write_html("../ghg_trend_top_10_high_impact.html")

fig.show()

In [ ]:
# Untersuchen warum Mining of coal and lignite einen peak in 2018 hat
# Filter erstellen
maske = (
    (df['Industry (Exiobase)'] == 'Mining of coal and lignite; extraction of peat (10)') &
    (df['Year'] == 2018))

# Gefilterte Daten anzeigen
df_mining_2018 = df[maske]
print(df_mining_2018)

# Alle Unternehmen im Gesamtzeitraum anzeigen, um mit Ausreißer zu vergleichen
maske = (df['Industry (Exiobase)'] == 'Mining of coal and lignite; extraction of peat (10)') 

df_mining_total = df[maske]
print(df_mining_total)

## Ausreißer ist auf ein Unternehnmen zurückzuführen -> möglicherweise durch neue Abbauprojekte
## 11 Mining company Einträge, davon 8 von einem Unternehmen